In [ ]:
import boto3
import pandas as pd
import psycopg2
import json

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('covid19-analytics.config'))

In [ ]:
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')


DWH_CLUSTER_TYPE = config.get('DWH', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH', 'DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH', 'DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER = config.get('DWH', 'DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH', 'DWH_DB')
DWH_DB_USER = config.get('DWH', 'DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH', 'DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH', 'DWH_PORT')
DWH_IAM_ROLE_NAME = config.get('DWH', 'DWH_IAM_ROLE_NAME')

TARGET_REGION = config.get('S3', 'TARGET_REGION')

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

In [ ]:
pd.DataFrame(
    {
        'Param':
            [
                'DWH_CLUSTER_TYPE',
                'DWH_NUM_NODES',
                'DWH_NODE_TYPE',
                'DWH_CLUSTER_IDENTIFIER',
                'DWH_DB',
                'DWH_DB_USER',
                'DWH_DB_PASSWORD',
                'DWH_PORT',
                'DWH_IAM_ROLE_NAME'
            ],
        'Value':
            [
                DWH_CLUSTER_TYPE,
                DWH_NUM_NODES,
                DWH_NODE_TYPE,
                DWH_CLUSTER_IDENTIFIER,
                DWH_DB,
                DWH_DB_USER,
                DWH_DB_PASSWORD,
                DWH_PORT,
                DWH_IAM_ROLE_NAME

            ]
    }
)

In [ ]:
iam_client = boto3.client(
                    'iam',
                    region_name=TARGET_REGION,
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [ ]:
redshift_client = boto3.client(
                    'redshift',
                    region_name=TARGET_REGION,
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

In [ ]:
roleArn = iam_client.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn

In [ ]:
try:
    response = redshift_client.create_cluster(
        # see doc: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift/client/create_cluster.html
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,

        # Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        Port=int(DWH_PORT),
        NumberOfNodes=int(DWH_NUM_NODES),
        Tags=[
            {'Key': 'ENVIRONMENT','Value': 'DEV'},
        ],

        # Roles (s3 access role)
        IamRoles=[roleArn],

        # Other settings
        MultiAZ=False,
    )
except Exception as e:
    print(e)

In [ ]:
# redshift_client.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)
